# SHUBHAM SHARMA
## IIT BOMBAY
This code is for prediction of predictions after we are done with *fusion network*

In [0]:
#For Deep learning
from keras.layers import LSTM, TimeDistributed , Dense, RepeatVector, Conv2D, MaxPooling2D, Input, BatchNormalization, Flatten, Embedding, Reshape, concatenate
import keras
from keras import callbacks 
from keras.models import Model
from keras.optimizers import adam,Adagrad,SGD

##
import numpy as np

Using TensorFlow backend.


In [0]:
def prediction_network(width, height, depth, verb_length = 1, num_verbs = 50, hidden_layer = 3500, encoding_nodes=4096, weightsPath=None):
  
  bn_flag = True
  inputs_to_vgg = Input((width, height , depth))
  conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs_to_vgg)
  conv1 = BatchNormalization(axis = -1)(conv1,training=bn_flag)
  conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
  conv1 = BatchNormalization(axis= -1)(conv1,training=bn_flag)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  ##
  conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
  conv2 = BatchNormalization(axis = -1)(conv2,training=bn_flag)
  conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
  conv2 = BatchNormalization(axis= -1)(conv2,training=bn_flag)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  ##
  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
  conv3 = BatchNormalization(axis = -1)(conv3,training=bn_flag)
  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
  conv3 = BatchNormalization(axis = -1)(conv3,training=bn_flag)
  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
  conv3 = BatchNormalization(axis = -1)(conv3,training=bn_flag)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  ##
  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
  conv4 = BatchNormalization(axis = -1)(conv4,training=bn_flag)
  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
  conv4 = BatchNormalization(axis = -1)(conv4,training=bn_flag)
  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
  conv4 = BatchNormalization(axis = -1)(conv4,training=bn_flag)
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
  ##
  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
  conv5 = BatchNormalization(axis = -1)(conv5,training=bn_flag)
  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
  conv5 = BatchNormalization(axis = -1)(conv5,training=bn_flag)
  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
  conv5 = BatchNormalization(axis = -1)(conv5,training=bn_flag)
  pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
  ##
  Y = Flatten()(pool5)
  Y = Dense(hidden_layer, activation='relu')(Y)
  Y = Dense(hidden_layer, activation='relu')(Y)
  Y = Dense(hidden_layer, activation='relu')(Y)
  Y = Reshape((1,hidden_layer))(Y)

  ####################################################################################################
  # Y is the output from the VGG network. Now let us make an output from the verb and concatenate them 
  input_verb = Input([verb_length])
  Z = Embedding(num_verbs, hidden_layer)(input_verb)
  
  input_to_lstm = concatenate([Z , Y],axis = 1)
  encoder = LSTM(encoding_nodes)(input_to_lstm)
  
  L = RepeatVector(39)(encoder)
  L = LSTM(2310 , return_sequences = True)(L)
  out = TimeDistributed(Dense(2310, activation = 'softmax'))(L)
  

  model = Model(inputs=[inputs_to_vgg , input_verb], outputs=[out])
  
  if weightsPath is not None:
    model.load_weights(weightsPath)
    
  print(model.summary())
  return model

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
X_images = np.load('gdrive/My Drive/Situation_recognition/data/X.npy')
X_verbs = np.load('gdrive/My Drive/Situation_recognition/data/Y_verbs.npy')
Y = np.load('gdrive/My Drive/Situation_recognition/data/Y_nouns_categorical.npy')

In [0]:
model = prediction_network(width = 256, height = 256, depth = 3, weightsPath='gdrive/My Drive/Situation_recognition/situation2.h5')












Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization_1[0][0]      
_________________________________________________________________________________

In [0]:
path2save = 'gdrive/My Drive/Situation_recognition/situation2.h5'
modelCheck = callbacks.ModelCheckpoint(path2save, monitor='loss', verbose=0, mode='auto')
 
opt = Adagrad(lr=1e-7)
#Set the compiler parameter for the training
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["categorical_accuracy"],sample_weight_mode='auto')
print ("Training the Model...")



Training the Model...


In [0]:
history = model.fit([X_images , X_verbs] , Y, batch_size=32, epochs=500, verbose=1, callbacks= [modelCheck], shuffle=True)#,validation_data = (x_test , y_test)
print ("Dumping Weights to file...")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/500
13379/13379 [==============================] - 274s 20ms/step - loss: 0.1585 - categorical_accuracy: 0.9723
Epoch 2/500
11552/13379 [========================>.....] - ETA: 35s - loss: 0.1597 - categorical_accuracy: 0.9722